In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import numpy as np 
import torch_geometric.nn as pyg_nn 
import torch_geometric.utils as pyg_utils
from torch_geometric.data import Dataset, Data, DataLoader
import torch.optim as optim
import os.path as osp
import scipy.io as sio
from dataset import QUASARDataset
from primal_model import PrimalModel
from primal_train import validate

In [2]:
test_dir_1 = '/home/hank/Datasets/QUASAR/small'
testset_1 = QUASARDataset(test_dir_1,num_graphs=100,remove_self_loops=True)
test_dir_2 = '/home/hank/Datasets/QUASAR/small-test'
testset_2 = QUASARDataset(test_dir_2,num_graphs=100,remove_self_loops=True)
test_dir_3 = '/home/hank/Datasets/QUASAR/N40-100'
testset_3 = QUASARDataset(test_dir_3,num_graphs=100,remove_self_loops=True)
test_dir_4 = '/home/hank/Datasets/QUASAR/N50-100'
testset_4 = QUASARDataset(test_dir_4,num_graphs=100,remove_self_loops=True)
train_dir = '/home/hank/Datasets/QUASAR/N30-1000'
trainset = QUASARDataset(train_dir,num_graphs=1000,remove_self_loops=True)
val_dir = '/home/hank/Datasets/QUASAR/N30-100'
valset = QUASARDataset(val_dir,num_graphs=100,remove_self_loops=True)
DEVICE = torch.device('cpu')

Data graph type: 1.
Data graph type: 1.
Data graph type: 1.
Data graph type: 1.


Processing...


Expected # graphs: 100. Actual # graphs: 100.
Data graph type: 1.
Data graph type: 1.


Done!


In [8]:
GNN_TYPE = 'SAGE'
GNN_HIDDEN_DIM = 64
GNN_OUT_DIM = GNN_HIDDEN_DIM
GNN_LAYER = 9
LR = 0.01
NODE_MODE = 1
DATA_GRAPH_TYPE = 1
NUM_EPOCHES = 1000
DROPOUT = 0.2
MLP_LAYER = 1
RESIDUAL = True
BATCHNORM = True
FACTOR = True
model = PrimalModel(
    node_feature_mode=NODE_MODE,
    gnn_type=GNN_TYPE,
    mp_hidden_dim=GNN_HIDDEN_DIM,mp_output_dim=GNN_OUT_DIM,mp_num_layers=GNN_LAYER, 
    primal_node_mlp_hidden_dim=GNN_OUT_DIM,primal_node_mlp_output_dim=10,
    node_mlp_num_layers=MLP_LAYER,
    primal_edge_mlp_hidden_dim=GNN_OUT_DIM,primal_edge_mlp_output_dim=10,
    edge_mlp_num_layers=MLP_LAYER,
    dropout_rate=DROPOUT,
    relu_slope=0.1,
    residual=RESIDUAL,
    batchnorm=BATCHNORM,
    factor=FACTOR)
print(model)
# model.load_state_dict(torch.load('./models/primal_model_N30-1000_SAGE_9_True_True_True_801.pth'))
# model.load_state_dict(torch.load('./models/primal_model_N30-1000_SAGE_5_True_True_True.pth'))
# model.load_state_dict(torch.load('./models/primal_model_N30-1000_SAGE_9_True_True_True.pth'))
# model.load_state_dict(torch.load('./models/primal_model_N30-1000_SAGE_15_True_True_True.pth'))
model.load_state_dict(torch.load('./models/primal_model_N30-1000_SAGE_9_64_2_True_True_True.pth'))
model.double()

train_acc = validate(model,trainset,DEVICE)
val_acc = validate(model,valset,DEVICE)
test_acc_1 = validate(model,testset_1,DEVICE)
test_acc_2 = validate(model,testset_2,DEVICE)
test_acc_3 = validate(model,testset_3,DEVICE)
test_acc_4 = validate(model,testset_4,DEVICE)
print('Train acc: {:.4f}.'.format(train_acc))
print('Val acc: {:.4f}.'.format(val_acc))
print('Test acc 1: {:.4f}.'.format(test_acc_1))
print('Test acc 2: {:.4f}.'.format(test_acc_2))
print('Test acc 3: {:.4f}.'.format(test_acc_3))
print('Test acc 4: {:.4f}.'.format(test_acc_4))

Model: node_feature_mode = 1, mp_input_dim = 6, relu_slope = 0.1. GNN type: SAGE. Residual: True. BatchNorm: True. Factor: True. Resmode: 1.
PrimalModel(
  (mp_convs): ModuleList(
    (0): SAGEConv(6, 64)
    (1): SAGEConv(64, 64)
    (2): SAGEConv(64, 64)
    (3): SAGEConv(64, 64)
    (4): SAGEConv(64, 64)
    (5): SAGEConv(64, 64)
    (6): SAGEConv(64, 64)
    (7): SAGEConv(64, 64)
    (8): SAGEConv(64, 64)
    (9): SAGEConv(64, 64)
    (10): SAGEConv(64, 64)
  )
  (primal_node_mlp): ModuleList(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=4, bias=True)
  )
  (bn_layers): ModuleList(
    (0): BatchNorm(64)
    (1): BatchNorm(64)
    (2): BatchNorm(64)
    (3): BatchNorm(64)
    (4): BatchNorm(64)
    (5): BatchNorm(64)
    (6): BatchNorm(64)
    (7): BatchNorm(64)
    (8): BatchNorm(64)
    (9): BatchNorm(64)
  )
)
Train acc: 0.4972.
Val acc: 0.6392.
Test acc 1: 0.6

Data(x=[41, 16], edge_index=[2, 1640], ud_edges=[820], edge_map=[41, 41], X=[164, 164], S=[164, 164], Aty=[164, 164], C=[164, 164])